# Cat Face GAN model  

This is a project to test some of the concepts I have learnt for a Generative Adversial Network (GAN) and also to practise using Git. 

## Step 1: Dataset Inspection 

Since the dataset I've obtained from https://github.com/Ferlix/Cat-faces-dataset should already be standardised to 29,842 64x64 RRB images of cats' faces, there shouldn't be a need to normalise them any further. However, it is still good practice to just take a look at the the pics just to make sure. 

In [18]:
import os 
from matplotlib.image import imread
import numpy as np

In [13]:
%cd "D:/data_science/kaggle/CatFaceGAN"
%pwd

D:\data_science\kaggle\CatFaceGAN


'D:\\data_science\\kaggle\\CatFaceGAN'

In [14]:
imgpath=r'dataset'

In [16]:
dimension1 = []
dimension2 = []
colours = []
for filename in os.listdir(imgpath):
    img = imread(imgpath + '\\' + filename)
    d1,d2,colour = img.shape
    dimension1.append(d1)
    dimension2.append(d2)
    colours.append(colour)

In [20]:
print (np.min(dimension1))
print (np.max(dimension1))
print (np.min(dimension2))
print (np.max(dimension2))
print (np.min(colours))
print (np.max(colours))
print(len(dimension1))

64
64
64
64
3
3
29843


As shown above, it is safe to assume that all 29,843 images are 64x64 and are RGB mode. 

## Step 2: creating GAN model

## Step 3: training GAN model 

In [11]:
from tensorflow.keras.preprocessing import image 
    for filename in os.listdir(imgpath + '\\' + filename)